In [1]:
#-*- coding : utf-8 -*-
# coding: utf-8

import numpy as np
import pandas as pd
raw_dataset = pd.read_csv('./datasets/adjustment.tsv', 
  sep='\t',
  skipinitialspace=True,encoding="utf-8")
  #skipinitialspace=True,encoding="gbk")
np.set_printoptions(precision=3, suppress=True)
dataset = raw_dataset.copy()
dataset.head()

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,7.4,0.048,1.19,1.47,0.23,0.54,0.27,5.75,0.65,0.71,...,0.07,0.62,1.23,0.88,0.73,0.19,1.67,0.36,1.58,1.62
1,3.7,NaN,1.87,0.18,0.94,0.70,0.07,5.20,0.73,0.59,...,0.91,0.89,0.99,1.11,0.13,0.18,1.33,0.86,1.22,1.30
2,1.2,0.046,1.97,1.17,0.67,0.89,0.60,2.10,0.97,0.14,...,0.80,0.69,1.85,0.13,1.54,1.91,0.63,0.70,1.71,1.57
3,7.2,0.053,1.17,2.13,0.82,0.18,0.17,5.30,0.27,0.46,...,0.37,0.37,1.72,0.58,1.03,1.77,0.86,1.65,1.95,1.37
4,1.8,0.021,1.89,0.42,0.20,0.92,0.80,1.60,0.20,0.70,...,0.50,0.68,0.83,0.09,0.94,0.49,1.87,1.21,1.53,1.56


In [2]:
print("剔除前：", dataset.shape)
#dataset=dataset.dropna()
#dataset = dataset.fillna(method='bfill').fillna(method='ffill')

print('特征5平均值=' + str(dataset['特征5'].mean()))
dataset['特征5'] = dataset['特征5'].fillna(value=dataset['特征5'].mean())
dataset = dataset.fillna(method='bfill').fillna(method='ffill')
dataset.iloc[:, 0:17]= (dataset.iloc[:, 0:17]- dataset.iloc[:, 0:17].min()) / (dataset.iloc[:, 0:17].max() - dataset.iloc[:, 0:17].min())

print("剔除后：", dataset.shape)
dataset.head()

剔除前： (4750, 26)
特征5平均值=0.496587016282512
剔除后： (4750, 26)


,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,0.74,0.48,0.19,0.49,0.23,0.54,0.27,0.95,0.65,0.71,...,0.07,0.62,1.23,0.88,0.73,0.19,1.67,0.36,1.58,1.62
1,0.37,0.46,0.87,0.06,0.94,0.70,0.07,0.84,0.73,0.59,...,0.91,0.89,0.99,1.11,0.13,0.18,1.33,0.86,1.22,1.30
2,0.12,0.46,0.97,0.39,0.67,0.89,0.60,0.22,0.97,0.14,...,0.80,0.69,1.85,0.13,1.54,1.91,0.63,0.70,1.71,1.57
3,0.72,0.53,0.17,0.71,0.82,0.18,0.17,0.86,0.27,0.46,...,0.37,0.37,1.72,0.58,1.03,1.77,0.86,1.65,1.95,1.37
4,0.18,0.21,0.89,0.14,0.20,0.92,0.80,0.12,0.20,0.70,...,0.50,0.68,0.83,0.09,0.94,0.49,1.87,1.21,1.53,1.56


In [3]:
average = np.average(dataset.values[:,:18], axis=0)
variance = np.var(dataset.values[:,:18], axis=0)
print('均值', average)
print('⽅差', variance)

均值 [0.495 0.504 0.509 0.5   0.501 0.497 0.506 0.498 0.5   0.498 0.497 0.494
 0.502 0.496 0.497 0.503 0.51  0.498]
⽅差 [0.084 0.082 0.084 0.084 0.082 0.085 0.084 0.084 0.084 0.082 0.083 0.083
 0.081 0.083 0.084 0.083 0.083 0.08 ]


In [4]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [5]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()
train_labels = train_features[['补偿'+str(i) for i in range(8)]].copy()
test_labels = test_features[['补偿'+str(i) for i in range(8)]].copy()
train_features = train_features.drop(['补偿'+str(i) for i in range(8)], axis=1)
test_features = test_features.drop(['补偿'+str(i) for i in range(8)], axis=1)
print(train_features.shape, train_labels.shape)

(3800, 18) (3800, 8)


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
print(tf.__version__)

2.6.0


In [7]:
import tensorflow as tf
from tensorflow.keras import layers
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [8]:
model = tf.keras.Sequential([ # 根据情况调整模型结构
layers.Dense(100, input_dim=train_features.shape[1], activation="softmax"),
layers.Dense(100, activation="relu"),
layers.Dense(32, activation="relu"),
layers.Dense(train_labels.shape[1])
])
model.compile(loss="mse", optimizer="Adam") # 根据情况调整参数
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1900      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                3232      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 264       
Total params: 15,496
Trainable params: 15,496
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit( # 根据情况调整参数
train_features, 
train_labels, 
epochs=30, 
batch_size=32
)

Epoch 1/30
119/119 [==============================] - 1s 975us/step - loss: 0.6075
Epoch 2/30
119/119 [==============================] - 0s 953us/step - loss: 0.2849
Epoch 3/30
119/119 [==============================] - 0s 975us/step - loss: 0.2848
Epoch 4/30
119/119 [==============================] - 0s 983us/step - loss: 0.2851
Epoch 5/30
119/119 [==============================] - 0s 976us/step - loss: 0.2850
Epoch 6/30
119/119 [==============================] - 0s 964us/step - loss: 0.2851
Epoch 7/30
119/119 [==============================] - 0s 990us/step - loss: 0.2846
Epoch 8/30
119/119 [==============================] - 0s 977us/step - loss: 0.2851
Epoch 9/30
119/119 [==============================] - 0s 975us/step - loss: 0.2846
Epoch 10/30
119/119 [==============================] - 0s 959us/step - loss: 0.2843
Epoch 11/30
119/119 [==============================] - 0s 952us/step - loss: 0.2845
Epoch 12/30
119/119 [==============================] - 0s 998us/step - loss: 0.2841
E

In [10]:
test_preds = model.predict(test_features)

In [11]:
print("y1 MSE:%.4f" % mean_squared_error(test_labels, test_preds))
# 直接删除 y1 MSE:0.2934
# 前差 y1 MSE:0.2902
# 后差 y1 MSE:0.2899，0.2927，0.2945  0.2872
# 先前再后 y1 MSE:0.2883，0.2901，0.2874，0.2890 第一个softmax = y1 MSE:0.2832,0.2830,0.2827,0.2822,0.2837
#        前两个softmax y1 MSE:0.2833,0.2837,0.2835,0.2837
# 先后再前 y1 MSE:0.2884，0.2880，0.2878，0.2863，0.2894，0.2865



y1 MSE:0.2823


In [12]:
tf.keras.models.save_model(
model,
#'/tf/models/adjustment/tensorflow/1/', # /tf/models/adjustment/tensorflow为tensorflow-serving的模型根⽬录
'./models/adjustment/1/',
overwrite=True,
include_optimizer=True,
save_format=None,
signatures=None,
options=None
)

INFO:tensorflow:Assets written to: ./models/adjustment/1/assets


In [13]:
import json
import requests
from pprint import pprint
req_data = json.dumps({
'inputs': test_features.values[:2].tolist(),
 }) 
pprint(req_data)
#response = requests.post('http://adjustment-serving-container:8501/v1/models/adjustment/versions/1:predict', # 根
response = requests.post('http://localhost:8501/v1/models/adjustment/versions/1:predict',
data=req_data,
headers={"content-type": "application/json"})
if response.status_code != 200:
  raise RuntimeError('Request tf-serving failed: ' + response.text)
resp_data = json.loads(response.text) 
if 'outputs' not in resp_data or type(resp_data['outputs']) is not list:
  raise ValueError('Malformed tf-serving response')
pprint(resp_data)

('{"inputs": [[0.74, 0.48, 0.18999999999999995, 0.49, 0.23, 0.54, 0.27, 0.95, '
 '0.65, 0.71, 0.09, 0.79, 0.09, 0.46, 0.82, 0.04, 0.07, 0.62], [0.72, '
 '0.5299999999999999, 0.16999999999999993, 0.71, 0.82, 0.18, 0.17, 0.86, 0.27, '
 '0.46, 0.85, 0.01, 0.41, 0.33, 0.59, 0.79, 0.37, 0.37]]}')


ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/adjustment/versions/1:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff1dd5a67c0>: Failed to establish a new connection: [Errno 61] Connection refused'))